Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');
df = convert(DataFrame, d);

dcc = filter(row -> !(ismissing(row[:rgdppc_2000])), df)
dcc[:log_gdp] = log.(dcc[:rgdppc_2000])
dcc[:cont_africa] = Array{Float64}(convert(Array{Int}, dcc[:cont_africa]))
dcc[:rugged] = convert(Array{Float64}, dcc[:rugged])
first(dcc[[:rugged, :cont_africa, :log_gdp]], 5)

,rugged,cont_africa,log_gdp
,Float64,Float64,Float64
1,0.858,1.0,7.49261
2,3.427,0.0,8.21693
3,0.769,0.0,9.93326
4,0.775,0.0,9.40703
5,2.688,0.0,7.79234


Define the Stan language model

In [4]:
m_8_1_model = "
data{
    int N;
    vector[N] log_gdp;
    vector[N] cont_africa;
    vector[N] rugged;
    vector[N] rugged_cont_africa;
}
parameters{
    real a;
    real bR;
    real bA;
    real bAR;
    real sigma;
}
model{
    vector[N] mu = a + bR * rugged + bA * cont_africa + bAR * rugged_cont_africa;
    sigma ~ uniform( 0 , 10 );
    bAR ~ normal( 0 , 10 );
    bA ~ normal( 0 , 10 );
    bR ~ normal( 0 , 10 );
    a ~ normal( 0 , 100 );
    log_gdp ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m_8_1_model",
monitors = ["a", "bR", "bA", "bAR", "sigma"],
model=m_8_1_model, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_8_1_model.stan will be updated.



Input data for cmdstan

In [6]:
m_8_1_data = Dict("N" => size(dcc, 1),
"log_gdp" => dcc[:log_gdp],  "rugged" => dcc[:rugged],
"cont_africa" => dcc[:cont_africa],
"rugged_cont_africa" => dcc[:rugged] .* dcc[:cont_africa] );

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_8_1_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.22 0.14  9.00  9.46   282    1
bR    -0.21 0.08 -0.33 -0.08   275    1
bA    -1.94 0.24 -2.33 -1.59   268    1
bAR    0.40 0.14  0.18  0.62   271    1
sigma  0.96 0.05  0.87  1.04   339    1
"

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -2.83688, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_8_1_model.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Inference for Stan model: m_8_1_model_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.36, 0.37, 0.37, 0.36) seconds, 1.5 seconds total
Sampling took (0.38, 0.40, 0.38, 0.40) seconds, 1.6 seconds total

                 Mean     MCSE  StdDev     5%    50%       95%    N_Eff  N_Eff/s    R_hat
lp__              -76  4.1e-02     1.7    -79    -75  -7.4e+01  1.6e+03  1.0e+03  1.0e+00
accept_stat__    0.9

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.22 0.14  9.00  9.46   282    1\nbR    -0.21 0.08 -0.33 -0.08   275    1\nbA    -1.94 0.24 -2.33 -1.59   268    1\nbAR    0.40 0.14  0.18  0.62   271    1\nsigma  0.96 0.05  0.87  1.04   339    1\n"

Describe the draws

In [8]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD        Naive SE       MCSE      ESS
    a  9.21992128 0.140932627 0.00222834049 0.0023330632 1000
   bR -0.20070737 0.078362018 0.00123901229 0.0014551924 1000
   bA -1.94886029 0.232665104 0.00367875831 0.0043123741 1000
  bAR  0.39216158 0.133758486 0.00211490735 0.0025426977 1000
sigma  0.94818026 0.052977286 0.00083764444 0.0011569341 1000

Quantiles:
          2.5%       25.0%       50.0%      75.0%       97.5%   
    a  8.94614825  9.12650750  9.2159500  9.31269000  9.49657450
   bR -0.36015257 -0.25171275 -0.1987345 -0.14808575 -0.04904898
   bA -2.40949000 -2.10836500 -1.9439700 -1.78687250 -1.50560800
  bAR  0.13470718  0.30143575  0.3881355  0.48148500  0.66009610
sigma  0.85288838  0.91057750  0.9459670  0.98318000  1.05894425



End of `08/m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*